In [22]:
import pandas as pd
from dotenv import load_dotenv
import os

os.environ.pop("API_KEY")
os.environ.pop("SECRET_KEY")

load_dotenv()

api_key = os.environ["API_KEY"]
api_secret = os.environ["SECRET_KEY"]

In [23]:
from binance import Client
client = Client(api_key, api_secret)

# Trading Strategy Report: Arbitrage between USDC and USDT

## 1. Executive Summary
### 1.1 Overview
This report presents an arbitrage trading strategy that exploits the price deviations between two stablecoins, USDC and USDT, on the Binance exchange. The strategy aims to buy USDC when its price is below 1 USDT and sell it when the price exceeds 1 USDT, and vice versa. The backtest, conducted over one year of historical data, reveals that the strategy is profitable under specific market conditions, though it is sensitive to trading fees and slippage.

## 2. Introduction
### 2.1 Background
USDC and USDT are two of the most widely used stablecoins, each pegged to the US dollar. Despite their pegged nature, small price deviations can occur due to market demand, liquidity conditions, and arbitrage opportunities. This strategy seeks to capitalize on these deviations.

### 2.2 Objective
The objective of this strategy is to generate profit by exploiting small, temporary price deviations between USDC and USDT on the Binance exchange.

## 3. Methodology
### 3.1 Data Collection
Historical price data for the USDC/USDT trading pair was collected from the Binance API. The data spans from August 1, 2023, to August 1, 2024, with a 1-minute granularity. The data was cleaned and formatted for analysis, with close prices being the primary focus for decision-making.

### 3.2 Strategy Description
The strategy is based on a simple arbitrage logic:

Buy USDC when the price of USDC/USDT is below 1, indicating USDC is undervalued relative to USDT.
Sell USDC when the price of USDC/USDT is above 1, indicating USDC is overvalued relative to USDT.
The reverse applies for USDT, buying it when it is undervalued and selling when it is overvalued.

### 3.3 Implementation
The strategy was implemented in Python using the Binance API for data retrieval. The backtest involved iterating over the historical data, executing buy and sell orders based on the strategy rules, and tracking the portfolio's value over time.


In [29]:
# Replace these with your Binance API key and secret key
# api_key = 'your_api_key_here'
# api_secret = 'your_secret_key_here'

# Initialize Binance API client
client = Client(api_key, api_secret)

# Fetch historical data
symbol = 'USDCUSDT'
start_time = '1 Aug, 2023'  # Modify as per your needs
end_time = '1 Aug, 2024'    # Modify as per your needs
interval = Client.KLINE_INTERVAL_1MINUTE  # 1 minute intervals

# Get historical klines (candlestick data)
klines = client.get_historical_klines(symbol, interval, start_time, end_time)

# Convert to DataFrame
df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 
                                   'close_time', 'quote_asset_volume', 'number_of_trades', 
                                   'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

# Clean up the data
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df['close'] = df['close'].astype(float)

In [30]:
# Initialize the portfolio and state
portfolio = {'USDT': 10000, 'USDC': 0}  # Start with 1000 USDT
current_position = None

df

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2023-08-01 00:00:00,1.00020000,1.00020000,1.00010000,1.0001,13474.00000000,1690848059999,13475.98530000,21,6379.00000000,6380.27580000,0
1,2023-08-01 00:01:00,1.00010000,1.00020000,1.00010000,1.0001,30536.00000000,1690848119999,30539.33170000,31,2781.00000000,2781.55620000,0
2,2023-08-01 00:02:00,1.00010000,1.00020000,1.00010000,1.0001,103355.00000000,1690848179999,103365.44790000,18,1124.00000000,1124.22480000,0
3,2023-08-01 00:03:00,1.00010000,1.00020000,1.00010000,1.0002,67225.00000000,1690848239999,67237.00840000,14,52859.00000000,52869.57180000,0
4,2023-08-01 00:04:00,1.00020000,1.00020000,1.00010000,1.0001,4378.00000000,1690848299999,4378.62070000,10,1829.00000000,1829.36580000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
527036,2024-07-31 23:56:00,1.00020000,1.00030000,1.00020000,1.0002,45234.00000000,1722470219999,45244.04160000,129,9948.00000000,9950.98440000,0
527037,2024-07-31 23:57:00,1.00030000,1.00030000,1.00020000,1.0002,175874.00000000,1722470279999,175913.22580000,82,40510.00000000,40522.15300000,0
527038,2024-07-31 23:58:00,1.00020000,1.00030000,1.00020000,1.0002,65074.00000000,1722470339999,65087.81720000,77,8024.00000000,8026.40720000,0
527039,2024-07-31 23:59:00,1.00020000,1.00030000,1.00020000,1.0003,32611.00000000,1722470399999,32619.80260000,94,22804.00000000,22810.84120000,0


In [31]:
# Backtest the strategy
for i in range(1, len(df)):
    price = df.loc[i, 'close']
    
    # Buy USDC when it's cheaper than 1 USDT
    if price < 1 and portfolio['USDT'] > 0:
        portfolio['USDC'] = portfolio['USDT'] / price
        portfolio['USDT'] = 0
        current_position = 'USDC'
        print(f"Bought USDC at {price} on {df.loc[i, 'timestamp']}")
    
    # Sell USDC when it's more expensive than 1 USDT
    elif price > 1 and portfolio['USDC'] > 0:
        portfolio['USDT'] = portfolio['USDC'] * price
        portfolio['USDC'] = 0
        current_position = 'USDT'
        print(f"Sold USDC at {price} on {df.loc[i, 'timestamp']}")

# Final portfolio value
final_value = portfolio['USDT'] + portfolio['USDC'] * df.loc[len(df)-1, 'close']
print(f"Final portfolio value: {final_value:.2f} USDT")


Bought USDC at 0.9999 on 2023-08-29 19:09:00
Sold USDC at 1.0001 on 2023-08-29 20:13:00
Bought USDC at 0.9999 on 2023-08-29 20:36:00
Sold USDC at 1.0001 on 2023-08-29 21:11:00
Bought USDC at 0.9999 on 2023-08-29 21:40:00
Sold USDC at 1.0001 on 2023-08-29 23:50:00
Bought USDC at 0.9999 on 2023-08-30 02:34:00
Sold USDC at 1.0001 on 2023-08-30 02:55:00
Bought USDC at 0.9999 on 2023-09-13 19:49:00
Sold USDC at 1.0001 on 2023-09-13 20:43:00
Bought USDC at 0.9999 on 2023-09-13 21:18:00
Sold USDC at 1.0001 on 2023-09-14 00:35:00
Bought USDC at 0.9999 on 2023-09-14 14:26:00
Sold USDC at 1.0001 on 2023-09-14 14:34:00
Bought USDC at 0.9999 on 2023-09-14 14:47:00
Sold USDC at 1.0001 on 2023-09-14 16:58:00
Bought USDC at 0.9999 on 2023-09-14 17:18:00
Sold USDC at 1.0001 on 2023-09-14 19:15:00
Bought USDC at 0.9999 on 2023-09-14 20:59:00
Sold USDC at 1.0001 on 2023-09-15 01:29:00
Bought USDC at 0.9999 on 2023-09-15 02:31:00
Sold USDC at 1.0001 on 2023-09-15 08:04:00
Bought USDC at 0.9999 on 2023-09

## 4. Results
### 4.1 Backtest Performance
The backtest revealed that the strategy executed 594 trades over the year. The final portfolio value was $10,615.18, representing a 6.15% increase from the initial capital of $10,000. 

